In [1]:
%matplotlib inline
import numpy as np                                    #arrays and matrix math
import pandas as pd                                   #work with DataFrames
import matplotlib.pyplot as plt                       #plotting and visualization
import matplotlib.dates as mdates                     #datetime formate in plots

import h5py                                           #import h5 files
import os                                             #OS operations
import scipy.signal as signal                         #signal processing
from scipy.io import loadmat                          #load MatLab m-files

from skimage.metrics import mean_squared_error as image_mse         #Mean Squared Error
from skimage.metrics import structural_similarity as image_ssim     #Structural Similarity Index
from skimage.metrics import peak_signal_noise_ratio as image_psnr   #Peak Signal-Noise Ratio

# Define arguments for text box in PLT.TEXT()
my_box = dict(boxstyle='round', facecolor='wheat', alpha=0.5)

# Check tensorflow GPU settings
import tensorflow as tf
sys_info = tf.sysconfig.get_build_info()
print('Tensorflow built with CUDA?',  tf.test.is_built_with_cuda())
print('Devices available:', tf.config.experimental.list_physical_devices())
print('# GPU available:', len(tf.config.experimental.list_physical_devices('GPU')))
print("CUDA: {} | cuDNN: {}".format(sys_info["cuda_version"], sys_info["cudnn_version"]))
tf.config.list_physical_devices()

Tensorflow built with CUDA? True
Devices available: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU')]
# GPU available: 1
CUDA: 64_101 | cuDNN: 64_7


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU')]

***

In [2]:
# EXPERIMENT 54 ONLY
data_exp54 = pd.read_pickle('data_exp54.pkl')
print('Experiment 54 shape: {}'.format(data_exp54.shape))

time_complete_exp54 = np.array([loadmat('time_complete_54.mat')['time']], dtype='datetime64[us]').reshape(-1)
print('Times for Experiment 54 shape: {}'.format(time_complete_exp54.shape))

trange_54 = mdates.date2num(time_complete_exp54)
xrange_54 = np.arange(4950, 5150)
print('Timerange 54: {}'.format(trange_54.shape))
print('X-range 54: {}'.format(xrange_54.shape))

Experiment 54 shape: (200, 4200000)
Times for Experiment 54 shape: (4200000,)
Timerange 54: (4200000,)
X-range 54: (200,)


In [ ]:
# fig, ax1 = plt.subplots(1, 1, figsize=(20,10))
# date_format = mdates.DateFormatter('%H:%M:%S')

# im1 = ax1.imshow(data_exp54, aspect='auto', cmap='seismic', vmin=-50, vmax=50, extent=[trange_54[0], trange_54[-1], xrange_54[-1], xrange_54[0]])
# ax1.xaxis_date(); ax1.xaxis.set_major_formatter(date_format)
# ax1.set_xlabel('Time [s] on 06/20/2019'); ax1.set_ylabel('Distance [m]'); ax1.set_title('Experiment 54')
# plt.colorbar(im1, ax=ax1)

# fig.autofmt_xdate()
# plt.show();

***

DWT (Discrete Wavelet Transform)

In [ ]:
from pywt import wavedec2 as DWT2 
from pywt import waverec2 as iDWT2 
from pywt import coeffs_to_array, array_to_coeffs

In [ ]:
def do_DWT(data, compression_ratio, wavelet='db1', level=3):
    data_exp = np.array(data) #make data np.array
    coeffs = DWT2(data_exp, wavelet=wavelet, level=level) #take the DWT
    coeff_arr, coeff_slices = coeffs_to_array(coeffs) #convert DWT coefficients to array
    coeff_sort = np.sort(np.abs(coeff_arr.reshape(-1))) #sort array of coefficients
    
    perc = compression_ratio
    threshold   = coeff_sort[int(np.floor((1-perc)*len(coeff_sort)))] #threshold nonrelevant coefficients
    coeff_arr_t = coeff_arr * (np.abs(coeff_arr) > threshold)
    coeffs_t    = array_to_coeffs(coeff_arr_t, coeff_slices, output_format='wavedec2') #take array to DWT coefficients
    data_idwt   = iDWT2(coeffs_t, wavelet=wavelet) #inverse DWT
    
    data_exp_norm  = data_exp  / np.max(np.abs(data_exp))  #normalize original data
    data_idwt_norm = data_idwt / np.max(np.abs(data_idwt)) #normalize reconstructed data  
    
    mse  = image_mse(data_exp_norm,  data_idwt_norm) #mse
    ssim = image_ssim(data_exp_norm, data_idwt_norm) #ssim
    psnr = image_psnr(data_exp_norm, data_idwt_norm) #psnr
    metrics = [mse, ssim, psnr]
    
    return data_idwt, coeff_arr_t, metrics

In [ ]:
data_idwt, coeff_arr_t, metrics = do_DWT(data_exp54, 0.10)

print('MSE: {:.3e}'.format(metrics[0]))
print('SSIM: {:.2f}'.format(metrics[1]*100))
print('PSNR: {:.2f}'.format(metrics[2]))

In [ ]:
def make_DWT_plots():
    fig, ax1 = plt.subplots(1, 1, figsize=(8,8))
    date_format = mdates.DateFormatter('%H:%M:%S')
    im1 = ax1.imshow(coeff_arr_t, aspect='auto', cmap='seismic', vmin=-50, vmax=50)
    ax1.set_xlabel('$Z_t$');        ax1.set_ylabel('$Z_d$')
    ax1.set_xticks([]);             ax1.set_yticks([])
    plt.show();
    
    fig, ax1  = plt.subplots(1, 1, figsize=(12,6))
    date_format = mdates.DateFormatter('%H:%M:%S')
    im1 = ax1.imshow(data_idwt, aspect='auto', cmap='seismic', vmin=-50, vmax=50, extent=[trange_54[0], trange_54[-1], xrange_54[-1], xrange_54[0]])
    ax1.xaxis_date(); ax1.xaxis.set_major_formatter(date_format)
    ax1.set_xlabel('Time [s] on 06/20/2019'); ax1.set_ylabel('Distance [m]') 
    ax1.set_title('DWT-Reconstructed Experiment 54')
    fig.autofmt_xdate()
    plt.show();

In [ ]:
make_DWT_plots()

***

# END